<a href="https://colab.research.google.com/github/Aravi9272/Expense-tracking-bot/blob/main/Expense_tracking_BOT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio spacy pandas matplotlib
!python -m spacy download en_core_web_sm
!apt-get install -y ffmpeg
!pip install SpeechRecognition



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 54.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is alr

In [ ]:
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta
import spacy
import os

nlp = spacy.load("en_core_web_sm")

expenses = []
next_id = 1
budgets = {}

def extract_amount(text):
    matches = re.findall(r'₹?\s?(\d+(?:,\d{3})*(?:\.\d{1,2})?)', text)
    if matches:
        return float(matches[0].replace(",", ""))
    return None

def extract_date(text):
    text = text.lower()
    today = datetime.today()

    if "day before yesterday" in text:
        return (today - timedelta(days=2)).date()
    if "yesterday" in text:
        return (today - timedelta(days=1)).date()
    if "today" in text:
        return today.date()
    if "last week" in text:
        return (today - timedelta(days=7)).date()

    match = re.search(r'(\d+)\s+(day|week|month|year)s?\s+ago', text)
    if match:
        num = int(match.group(1))
        unit = match.group(2)
        if unit == "day":
            return (today - timedelta(days=num)).date()
        elif unit == "week":
            return (today - timedelta(weeks=num)).date()
        elif unit == "month":
            return (today - timedelta(days=30*num)).date()
        elif unit == "year":
            return (today - timedelta(days=365*num)).date()

    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "DATE":
            try:
                parsed = pd.to_datetime(ent.text, errors='coerce')
                if pd.notnull(parsed):
                    return parsed.date()
            except:
                pass

    return today.date()

def extract_category(text):
    text = text.lower()
    # look for 'on <category>', 'for <category>', etc.
    match = re.search(r'(?:on|for|to|spent|bought|paid)\s+(\w+)', text)
    if match:
        return match.group(1).lower()
    return "other"

def get_expense_by_id(eid):
    for e in expenses:
        if e['id'] == eid:
            return e
    return None

def delete_expense_by_id(eid):
    global expenses
    expenses = [e for e in expenses if e['id'] != eid]

def edit_expense(eid, new_amount=None, new_category=None, new_date=None):
    e = get_expense_by_id(eid)
    if e:
        if new_amount is not None:
            e['amount'] = new_amount
        if new_category is not None:
            e['category'] = new_category
        if new_date is not None:
            e['date'] = new_date
        return True
    return False


def summarize_compare_prev_month():
    if not expenses:
        return "No expenses yet."

    df = pd.DataFrame(expenses)
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.to_period('M')

    current_month = df['month'].max()
    prev_month = current_month - 1

    current_sum = df[df['month'] == current_month]['amount'].sum()
    prev_sum = df[df['month'] == prev_month]['amount'].sum()

    diff = current_sum - prev_sum
    if diff > 0:
        return f"You spent ₹{diff:.2f} more this month compared to previous month."
    elif diff < 0:
        return f"You spent ₹{-diff:.2f} less this month compared to previous month."
    else:
        return "Your spending is the same as last month."

def process_nl_query(text):
    text = text.lower()
    amount = 0
    category = None
    timeframe = None

    cats = set([e['category'] for e in expenses])
    for cat in cats:
        if cat in text:
            category = cat
            break

    if "last week" in text:
        start = datetime.today() - timedelta(days=7)
        timeframe = (start.date(), datetime.today().date())
    elif "last month" in text:
        today = datetime.today()
        start = (today.replace(day=1) - timedelta(days=1)).replace(day=1)
        end = today.replace(day=1) - timedelta(days=1)
        timeframe = (start.date(), end.date())
    elif "this month" in text:
        today = datetime.today()
        start = today.replace(day=1)
        timeframe = (start.date(), today.date())
    else:
        timeframe = (datetime(2000,1,1).date(), datetime.today().date())

    filtered = expenses
    if category:
        filtered = [e for e in filtered if e['category'] == category]
    if timeframe:
        filtered = [e for e in filtered if timeframe[0] <= e['date'] <= timeframe[1]]

    amount = sum(e['amount'] for e in filtered)
    return f"Total spent on **{category if category else 'all categories'}** from {timeframe[0]} to {timeframe[1]} is ₹{amount:.2f}."

def process_message(message, history):
    global next_id

    message_lower = message.lower()

    # Set budget command
    m = re.match(r'set budget ₹?(\d+(?:,\d{3})*(?:\.\d{1,2})?) for (\w+)', message_lower)
    if m:
        amount = float(m.group(1).replace(',', ''))
        category = m.group(2)
        set_budget(category, amount)
        reply = f" Budget set to ₹{amount:.2f} for category **{category}**."
        history.append([message, reply])
        return history

    # Delete expense by id
    m = re.match(r'delete expense (\d+)', message_lower)
    if m:
        eid = int(m.group(1))
        if get_expense_by_id(eid):
            delete_expense_by_id(eid)
            reply = f" Expense with ID {eid} deleted."
        else:
            reply = f" No expense found with ID {eid}."
        history.append([message, reply])
        return history

    # Edit expense amount by id
    m = re.match(r'edit expense (\d+) to ₹?(\d+(?:,\d{3})*(?:\.\d{1,2})?)', message_lower)
    if m:
        eid = int(m.group(1))
        new_amount = float(m.group(2).replace(',', ''))
        if edit_expense(eid, new_amount=new_amount):
            reply = f" Expense with ID {eid} updated to ₹{new_amount:.2f}."
        else:
            reply = f" No expense found with ID {eid}."
        history.append([message, reply])
        return history

    # Natural language query
    if message_lower.startswith("how much") or message_lower.startswith("what is") or message_lower.startswith("show me"):
        reply = process_nl_query(message)
        history.append([message, reply])
        return history

    # Otherwise assume expense entry
    amount = extract_amount(message)
    if amount is None:
        reply = " Couldn't detect a valid amount. Please try again."
        history.append([message, reply])
        return history

    category = extract_category(message)
    date = extract_date(message)

    expenses.append({
        'id': next_id,
        'amount': amount,
        'category': category,
        'date': date,
        'note': ''
    })
    reply = f"₹{amount:.2f} added to **{category}** on {date}. Expense ID: {next_id}"
    next_id += 1

    history.append([message, reply])
    return history

def generate_summary():
    if not expenses:
        return "No expenses yet.", None, None, None, None

    df = pd.DataFrame(expenses)
    df['date'] = pd.to_datetime(df['date'])
    df['month'] = df['date'].dt.to_period('M')

    # Category-wise bar chart
    cat_fig, ax1 = plt.subplots()
    df.groupby('category')['amount'].sum().sort_values().plot(kind='barh', ax=ax1, color='skyblue')
    ax1.set_title('Spending by Category')
    ax1.set_xlabel('Amount (₹)')
    plt.tight_layout()

    # Daily spending line chart
    daily_fig, ax2 = plt.subplots()
    df.groupby(df['date'].dt.date)['amount'].sum().plot(ax=ax2, marker='o', linestyle='-', color='orange')
    ax2.set_title('Daily Spending Trend')
    ax2.set_ylabel('Amount (₹)')
    ax2.set_xlabel('Date')
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Pie chart category share
    pie_fig, ax3 = plt.subplots()
    df.groupby('category')['amount'].sum().plot.pie(ax=ax3, autopct='%1.1f%%', startangle=90, colormap='tab20')
    ax3.set_ylabel('')
    ax3.set_title('Spending Share by Category')
    plt.tight_layout()

    # Budget usage bar
    current_month = df['month'].max()
    monthly_total = df[df['month'] == current_month]['amount'].sum()
    budget = sum(budgets.get(cat, 0) for cat in budgets)
    percent = min((monthly_total / budget) * 100, 100) if budget > 0 else 0

    budget_fig, ax4 = plt.subplots()
    ax4.barh(['Budget Used'], [percent], color='green' if percent < 100 else 'red')
    ax4.set_xlim(0, 100)
    ax4.set_title(f'Budget Usage ({monthly_total:.0f} / ₹{budget})')
    for i, v in enumerate([percent]):
        ax4.text(v + 1, i, f"{v:.1f}%", va='center')
    plt.tight_layout()

    total = df['amount'].sum()
    compare_text = summarize_compare_prev_month()

    summary = f" Total Spent: ₹{total:.2f}\n Last Month Spent Comparison:\n{compare_text}"

    return summary, cat_fig, daily_fig, pie_fig, budget_fig

def export_csv():
    if not expenses:
        return None
    df = pd.DataFrame(expenses)
    path = "expenses_export.csv"
    df.to_csv(path, index=False)
    return path



def respond(user_message, chat_history):
    updated_history = process_message(user_message, chat_history)
    return updated_history, ""
import speech_recognition as sr
def respond_voice(audio_path, chat_history):
    if audio_path is None:
        return chat_history, ""
    r = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = r.record(source)
    try:
        text = r.recognize_google(audio)
    except:
        text = ""
    if text == "":
        chat_history.append(["(voice input failed)", "Could not recognize speech, please try typing."])
        return chat_history, ""
    else:
        chat_history.append([f"(voice) {text}", "Processing..."])
        updated_history = process_message(text, chat_history)
        return updated_history, ""

def export_csv_func():
    path = export_csv()
    if path is None:
        return None
    return path

with gr.Blocks() as demo:
    gr.Markdown("Expense Tracking chatbot")
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Enter your expense details here...")
    submit = gr.Button("Send")
    voice_input = gr.Audio(type="filepath", label="Or speak your expense")


    export_btn = gr.Button("Export Expenses to CSV")
    export_output = gr.File(label="Download CSV here")

    with gr.Column():
        summary_btn = gr.Button("Show Summary & Visualizations")
        summary_text = gr.Textbox(label="Summary", interactive=False, lines=4)
        cat_plot = gr.Plot(label="Spending by Category")
        daily_plot = gr.Plot(label="Daily Spending Trend")
        pie_plot = gr.Plot(label="Spending Share by Category")
        budget_plot = gr.Plot(label="Budget of category")

    history = gr.State([])

    submit.click(respond, inputs=[msg, history], outputs=[chatbot, msg])
    voice_input.change(respond_voice, inputs=[voice_input, history], outputs=[chatbot, msg])
    export_btn.click(export_csv_func, inputs=None, outputs=export_output)

    def update_summary():
        s, c, d, p, b = generate_summary()
        return s, c, d, p, b

    summary_btn.click(update_summary, inputs=None, outputs=[summary_text, cat_plot, daily_plot, pie_plot, budget_plot])

demo.launch()


<ipython-input-3-0d9fc777e507>:308: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d00daf21454473cc01.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
